<a href="https://colab.research.google.com/github/abhinavdayal/EVA4/blob/master/S4/EVA4_Session_2_WO_Bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# call the heavenly gods
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=1, bias=False) #input 28x28x1, output 28x28x10, RF 3
        self.bn1 = nn.BatchNorm2d(10)
        self.d1 = nn.Dropout(0.05) #5 % probability 

        self.conv2 = nn.Conv2d(10, 10, 3, padding=1, bias=False) #input 28x28x10, output 28x28x10, RF 5
        self.bn2 = nn.BatchNorm2d(10)
        self.d2 = nn.Dropout(0.05) #5 % probability 

        self.pool1 = nn.MaxPool2d(2, 2) # RF 6

        self.conv3 = nn.Conv2d(10, 16, 3, padding=1, bias=False) #input 14x14x10, output 14x14x16, RF 10
        self.bn3 = nn.BatchNorm2d(16)
        self.d3 = nn.Dropout(0.05) #5 % probability 

        self.conv4 = nn.Conv2d(16, 16, 3, padding=1, bias=False) #input 14x14x16, output 14x14x16, RF 14
        self.bn4 = nn.BatchNorm2d(16)
        self.d4 = nn.Dropout(0.05) #5 % probability 

        self.pool2 = nn.MaxPool2d(2, 2) # RF 16
        
        self.conv5 = nn.Conv2d(16, 32, 3, padding=1, bias=False) #input 7x7x16, output 7x7x32, RF 24
        self.bn5 = nn.BatchNorm2d(32)
        self.d5 = nn.Dropout(0.05) #5 % probability 

        self.conv6 = nn.Conv2d(32, 32, 3, bias=False) #input 7x7x32, output 5x5x32, RF 32
        self.bn6 = nn.BatchNorm2d(32)
        self.d6 = nn.Dropout(0.05) #5 % probability 
        
        self.conv7 = nn.Conv2d(32, 10, 1, bias=False) #input 5x5x32, output 5x5x10, RF 32

    def forward(self, x):
        x = self.pool1(F.relu(self.bn2(self.d2(self.conv2(F.relu(self.bn1(self.d1(self.conv1(x)))))))))
        x = self.pool2(F.relu(self.bn4(self.d4(self.conv4(F.relu(self.bn3(self.d3(self.conv3(x)))))))))
        x = self.conv7(F.relu(self.bn6(self.d6(self.conv6(F.relu(self.bn5(self.d5(self.conv5(x)))))))))
        x = F.avg_pool2d(x, 5)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [16]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]              90
           Dropout-2           [-1, 10, 28, 28]               0
       BatchNorm2d-3           [-1, 10, 28, 28]              20
            Conv2d-4           [-1, 10, 28, 28]             900
           Dropout-5           [-1, 10, 28, 28]               0
       BatchNorm2d-6           [-1, 10, 28, 28]              20
         MaxPool2d-7           [-1, 10, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           1,440
           Dropout-9           [-1, 16, 14, 14]               0
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 14, 14]           2,304
          Dropout-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
        MaxPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    pbar =  tqdm_notebook(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    train_loss /= len(train_loader.dataset)

    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in tnrange(1, 21):
    print("\nEpoch:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train set: Average loss: 0.5462, Accuracy: 51684/60000 (86.14%)
Test set: Average loss: 0.1420, Accuracy: 9588/10000 (95.88%)

Epoch: 2


Train set: Average loss: 0.0831, Accuracy: 58701/60000 (97.83%)
Test set: Average loss: 0.0707, Accuracy: 9795/10000 (97.95%)

Epoch: 3


Train set: Average loss: 0.0567, Accuracy: 59057/60000 (98.43%)
Test set: Average loss: 0.0481, Accuracy: 9862/10000 (98.62%)

Epoch: 4


...

Train set: Average loss: 0.0186, Accuracy: 59654/60000 (99.42%)
Test set: Average loss: 0.0218, Accuracy: 9926/10000 (99.26%)

Epoch: 17


Train set: Average loss: 0.0195, Accuracy: 59650/60000 (99.42%)
Test set: Average loss: 0.0184, Accuracy: 9944/10000 (99.44%)

Epoch: 18


Train set: Average loss: 0.0191, Accuracy: 59624/60000 (99.37%)
Test set: Average loss: 0.0216, Accuracy: 9925/10000 (99.25%)

Epoch: 19


Train set: Average loss: 0.0176, Accuracy: 59674/60000 (99.46%)
Test set: Average loss: 0.0209, Accuracy: 9943/10000 (99.43%)

Epoch: 20


Train set: Average loss: 0.0177, Accuracy: 59654/60000 (99.42%)
Test set: Average loss: 0.0196, Accuracy: 9936/10000 (99.36%)
